# Serviceability limit state

Application example of SCI P354, *Design of Floors for Vibration* and comparison against a Footfall analysis with Robot Structural Analysis, as well as vibration results with Calculatis by Stora Enso.

## Simply supported uni-directional floor

This example consists on a simply supported uni-directional concrete floor on which a person is walking. The floor is analyzed as a one meter width simply supported beam and the walker's weight is distributed over the total floor width.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Analysis definitions

In [ ]:
t_end = 8  # s
sampling_points = 2000
gravity = 9.81  # m/s^2
t = np.linspace(0, t_end, sampling_points)
dt = t_end / sampling_points

## Structure properties

Using Rayleigh's method with an appropriate shape function corresponding for the first mode of vibration,
$$
\psi = \sin\left(\frac{\pi x}{L}\right)
$$
the modal stifness, mass and frequency read
$$
k = \int_0^L EI\psi''^2 dx = \frac{\pi^4EI}{2L^3} \quad \text{m/N per m width} \\
m = \int_0^L \rho A\psi^2 dx = \frac{\rho AL}{2} \quad \text{kg per m width} \\
\omega = \sqrt{\frac{k}{m}} = \frac{\pi^2}{L^2}\sqrt{\frac{EI}{\rho A}} \quad \text{rad/s}
$$

In [ ]:
class Structure:

    floor_span = 6  # m
    floor_width = 1  # m
    floor_thickness = 0.3  # m
    imposed_load = 4000  # N/m^2
    material_density = 2400  # kg/m^3
    Young_modulus = 2.7e10  # Pa (concrete)
    damping = 0.05  # %
    
    @property
    def mass(self):
        linear_density = self.material_density * self.floor_thickness + self.imposed_load / gravity * self.floor_width  # kg/m
        return 0.5 * linear_density * self.floor_span  # kg per m width
    
    @property
    def stiffness(self):
        inertia = 1 / 12 * self.floor_thickness**3  # m^4 per m width
        return 0.5 * np.pi**4 * self.Young_modulus * inertia / self.floor_span**3 * self.floor_width  # m/N

    @property
    def frequency(self):
        return np.sqrt(self.stiffness / self.mass)  # rad/s

structure = Structure()
print(f'Floor frequency: {0.5*structure.frequency/np.pi:.2f} Hz')

## Excitation

The nature of the excitation can be of different natures: walking, rythmic, indivudual or group. In this example, a single person walking is considered, which is the most common case for a residential area. The exitation function is defined using the a Fourier series decomposition, where the coefficients are defined in Table 3.1 of SCI P354.

Finally, the total time where the force is applied depends on the walking path length and the walking velocity. The walking velocity depends on the frequency according to the expression
$$
v = 1.67f^2 - 4.83*f + 4.5
$$

In [ ]:
class Action:

    walker_weight = 76  # kg
    natural_walking_frequency = 2  # Hz
    walking_frequency = 2 * np.pi * natural_walking_frequency  # rad/s
    walking_frequencies = walking_frequency * np.arange(1, 5, 1)
    walking_velocity = 1.67 * natural_walking_frequency**2 - 4.83 * natural_walking_frequency + 4.5  # m/s
    walking_path_length = 5  # m
    walking_time = walking_path_length / walking_velocity
    
    @property
    def Fourier_coef(self):
        return [
            0.436 * (1 * self.natural_walking_frequency - 0.95),
            0.006 * (2 * self.natural_walking_frequency + 12.3),
            0.007 * (3 * self.natural_walking_frequency + 5.20),
            0.007 * (4 * self.natural_walking_frequency + 2.00)]

    @property
    def phase_lag(self):
        return [0, -0.5*np.pi, np.pi, 0.5*np.pi]

    @property
    def force(self):
        return sum(self.walker_weight * gravity * a * np.sin(f * t + p) for a, f, p in zip(self.Fourier_coef, self.walking_frequencies, self.phase_lag))
    
    @property
    def force_total(self):
        return sum([self.force, self.walker_weight * gravity * np.ones(sampling_points)])

action = Action()
print(f'Walking time: {action.walking_time:.2f} s')
print(f'Walking frequency: {action.natural_walking_frequency:.2f} Hz')
print(f'Walking velocity: {action.walking_velocity:.2f} m/s')
print(f'Fourier terms: {", ".join(f"{a:.2f}" for a in action.Fourier_coef)}')

In [ ]:
plt.figure(figsize=(15,3))
plt.subplot(1,2,1)
plt.plot(t, action.force)
plt.xlabel('Time ($s$)')
plt.ylabel('Force ($N$)')
plt.title('Dynamic component of the excitation (4 Fourier terms)')
plt.subplot(1,2,2)
plt.plot(t, action.force_total)
plt.plot([0, t_end], [action.walker_weight*gravity, action.walker_weight*gravity], color='black', lw=.5)
plt.ylim(bottom=0)
plt.xlabel('Time ($s$)')
plt.ylabel('Force ($N$)')
plt.title('Total excitation force (4 Fourier terms)')
plt.show()

## Weighting function

The weighting function represents the human sensitivity for the perception of different frequencies. This empirical function has the following form:

![image](img/weighting_curve_Wb.png)

$$
W_b = \begin{cases}
0.4 & 1 < f < 2 \ \text{Hz} \\
f/5 & 2 < f < 5 \ \text{Hz} \\
1.0 & 5 < f < 16 \ \text{Hz} \\
16/f & f > 16 \ \text{Hz}
\end{cases}
$$

Since the dynamic response of the excitation is the superposition of multiple frequencies, rather than multiplying the acceleration response by the weighting function, here we will multiply each component of the force by the corresponding weigthing value.

In [ ]:
class WeightedAction(Action):

    @staticmethod
    def Wb(frequency):
        nat_f = 0.5 * frequency / np.pi
        if nat_f < 2:
            return 0.4
        elif nat_f < 5:
            return 0.2 * nat_f
        elif nat_f < 16:
            return 1.0
        else:
            return 16.0 / nat_f

    @property
    def force(self):
        return sum(self.walker_weight * gravity * a * np.sin(f * t + p) * self.Wb(f) for a, f, p in zip(self.Fourier_coef, self.walking_frequencies, self.phase_lag))

action = WeightedAction()

## Stationary dynamic respone

The stationary (or resonant) response is obtained multiplying the static response by the dynamic magnification factor H and the weighting function W. After substitution, the starionary acceleration reads:

$$
a_{rms} = \psi_{e,n} \psi_{r,n} \frac{F a_h}{m_n \sqrt{2}}H_{a,h,n}W_{b,h}
$$

Where the subindex $h$ stands for the $i^{th}$ harmonic and $n$ stands for the $i^{th}$ mode of vibration. $\psi_e$ and $\psi_r$ are the shape functions evaluated at corresponding the excitation and response points. For this particular case, both excitation and response points are at the midspan and $\psi_e=\psi_r=1$. Finally, the dynamic magnification factor for accelerations is defined as

$$
H_a = \frac{\gamma^2}{\sqrt{(1-\gamma^2)^2 + 4\xi^2\gamma^2}}
$$

with $\gamma=\Omega_h/\omega_n$, where $\Omega_h$ is the excitation frequency for the $h^{th}$ harmonic and $\omega_n$ is the response frequency for the $n^{th}$ mode.

In [ ]:
class StationaryResponse():

    def __init__(self, structure, action):
        accelerations = []
        for a, f in zip(action.Fourier_coef, action.walking_frequencies):
            g = f / structure.frequency
            Ha = g**2 / np.sqrt((1 - g**2)**2 + 4 * structure.damping**2 * g**2)
            acc = a * action.walker_weight * gravity / structure.mass * Ha * action.Wb(f)
            accelerations.append(acc)
        self.a_peak = sum(accelerations)

## Impulsive response

When the natural frequency is much greater than the walking frequency ($f_1 > 4f_w$), the equivalent impulse is defined by the following empirical formula (SCI P354, Eq (18)):
$$
F_I = 60\frac{f_w^{1.43}}{f_1^{1.3}}\frac{Q}{700}
$$
where $f_w$ is the natural walking frequency, $f_1$ is the first natural frequency and $Q$ is an 'average person' weight, taken as $760N$.

Finally, the peak acceleration response for an impulsive excitation is
$$
a_{I,peak} = \omega_1 \frac{F_I}{m_1}
$$
This value may be multiplied by appropriate shape functions $\psi$ and weighting function W.

In [ ]:
class ImpulsiveResponse():

    def __init__(self, structure, action):
        nat_fh = 0.5 * action.walking_frequency / np.pi
        nat_f1 = 0.5 * structure.frequency / np.pi
        impulse = 60 * nat_fh**1.43 / nat_f1**1.3 * action.walker_weight * gravity / 700
        self.a_peak = structure.frequency * impulse / structure.mass
        self.acc = self.a_peak * np.sin(structure.frequency*t) * np.exp(-structure.damping*structure.frequency*t)
        self.T = 2 * np.pi / action.walking_frequency
        self.a_rms = np.sqrt(np.mean(self.acc[:np.searchsorted(t, self.T)]**2))
        self.RF = self.a_rms / 0.005

imp = ImpulsiveResponse(structure, action)
plt.figure(figsize=(15,5))
plt.plot(t, imp.acc, label='Impulsive response')
plt.plot([0, t_end], [imp.a_peak, imp.a_peak], color='orange', label=f'Peak acceleration = {100*imp.a_peak:.2f} $cm/s^2$')
plt.plot([0, t_end], [-imp.a_peak, -imp.a_peak], color='orange')
plt.plot([0, imp.T], [imp.a_rms, imp.a_rms], color='orange', ls='--', label=f'RMS = {100*imp.a_rms:.2f} $cm/s^2$')
plt.plot([0, imp.T], [-imp.a_rms, -imp.a_rms], color='orange', ls='--')
plt.axvspan(0, imp.T, color='gainsboro', label='Structure period')
plt.text(0.5*imp.T, imp.a_rms+(imp.a_peak/10), f'RF = {imp.RF:.1f}', bbox=dict(alpha=.8, ec='gray', fc='white'))
plt.title('Impulsive response')
plt.xlabel('Time ($s$)')
plt.ylabel('Acceleration ($m/s^2$)')
plt.legend(framealpha=.95)
plt.show()

## Transient dynamic response
The transient dynamic response can be obtained by means of the Duhamel convolution
$$
u(t) = \frac{1}{m\omega} \int_0^t F(\tau) u_{free}(t-\tau) d\tau
$$
where the free response is given by
$$
u_{free}(t) = e^{-\xi\omega t}\sin(\omega t)
$$

In [ ]:
class TransientResponse():

    def __init__(self, structure, action):
        self.free = np.exp(-structure.damping * structure.frequency * t) * np.sin(structure.frequency * t)
        u_transient = np.convolve(action.force, self.free, 'full') * dt / structure.mass / structure.frequency
        u_transient = u_transient[:len(u_transient)//2+1]
        self.acc = np.diff(u_transient, 2) / dt**2
        self.acc = np.append(self.acc, self.acc[-1])
        self.acc = np.append(self.acc, self.acc[-1])
        walking_end_idx = np.searchsorted(t, action.walking_time)
        self.a_rms = self.acc[:walking_end_idx]
        self.ressonance_fct = 1 -np.exp(-structure.damping * structure.frequency * t)
        self.a_rms = np.sqrt(np.mean((self.a_rms*self.ressonance_fct[:walking_end_idx])**2))
        self.a_peak = max(self.acc)
        self.RF = self.a_rms / 0.005

stat = StationaryResponse(structure, action)
trans = TransientResponse(structure, action)

plt.figure(figsize=(15,5))
plt.plot(t, trans.acc, label='Transient')
plt.plot(t, trans.ressonance_fct * stat.a_peak, 'black', lw=.5, label='Ressonance bulid-up factor')
plt.plot([0, t_end], [stat.a_peak, stat.a_peak], 'orange', label=f'Stationary peak acceleration = {100*stat.a_peak:.2f} $cm/s^2$')
plt.plot([0, t_end], [-stat.a_peak, -stat.a_peak], 'orange')
plt.plot([0, action.walking_time], [trans.a_rms, trans.a_rms], 'orange', ls='--', label=f'Transient RMS acceleration = {100*trans.a_rms:.2f} $cm/s^2$')
plt.plot([0, action.walking_time], [-trans.a_rms, -trans.a_rms], 'orange', ls='--')
plt.plot(t, stat.a_peak * trans.free, color='maroon', lw=1, label='Free response')
plt.text(0.5*action.walking_time, trans.a_rms+(trans.a_peak/10), f'RF = {trans.RF:.1f}', bbox=dict(alpha=.8, ec='gray', fc='white'))
plt.axvspan(0, action.walking_time, color='gainsboro', label='Walking path length (t=Lp/v)')
plt.title('Transient response')
plt.xlabel('Time ($s$)')
plt.ylabel('Acceleration ($m/s^2$)')
plt.legend(loc='upper right', framealpha=.95)
plt.show()

In [ ]:
loads = range(0,40000,100)
imp_a = []
stat_a = []
trans_a = []
for imposed_load in loads:
    structure = Structure()
    structure.imposed_load = imposed_load
    action = WeightedAction()
    imp = ImpulsiveResponse(structure, action)
    stat = StationaryResponse(structure, action)
    trans = TransientResponse(structure, action)
    imp_a.append(imp.a_peak)
    stat_a.append(stat.a_peak)
    trans_a.append(trans.a_peak)

plt.plot(loads, imp_a, label='Impulsive response')
plt.plot(loads, stat_a, label='Stationary response')
plt.plot(loads, trans_a, label='Transient response', color='orange', ls=':')

from scipy.signal import argrelmax
harmonics = argrelmax(np.array(stat_a))[0]
for i, h in enumerate(harmonics):
    plt.plot(loads[h], stat_a[h], marker='o', ls='None', color='k')
    plt.text(loads[h], stat_a[h]+.005, f'Harmonic {4-i}')

plt.xlabel('Imposed load ($N$)')
plt.ylabel('Peak acceleration ($m/s^2$)')
plt.legend()
plt.show()